## Libraries

In [ ]:
import os
import glob
import shutil
import keras
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

import matplotlib.pyplot as plt

### Download the dataset

In [ ]:
### Login to your google account and,
## From tensorflow website download the flower_photos.tgz dataset

def get_dataset():
    _URL = "# tensorflow dataset link/flower_photos.tgz"
    zip_file = tf.keras.utils.get_file(origin=_URL, fname="flower_photos.tgz", extract=True)
    base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')
    
    return base_dir

base_dir = get_dataset()

#### Defining the classes in the dataset

In [ ]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

### Preparing the train and val datasets

In [ ]:
def prepare_dataset(base_dir):
    
    global classes
    for _class in classes:
        img_path = os.path.join(base_dir, _class)
        images = glob.glob(img_path + '/*.jpg')
        print("{}: {} Images".format(_class, len(images)))
        train, val = images[:round(len(images)*0.8)], images[round(len(images)*0.8):]

        for t in train:
            if not os.path.exists(os.path.join(base_dir, 'train', _class)):
                os.makedirs(os.path.join(base_dir, 'train', _class))
            shutil.move(t, os.path.join(base_dir, 'train', _class))
        for v in val:
            if not os.path.exists(os.path.join(base_dir, 'val', _class)):
                os.makedirs(os.path.join(base_dir, 'val', _class))
            shutil.move(v, os.path.join(base_dir, 'val', _class))
    
    return train, val

In [ ]:
train, val = prepare_dataset(base_dir)

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

### ImageDataGenerator for Image Augmentation

In [ ]:
img_shape = 150

def define_image_generators(train_dir, val_dir):
    
    global img_shape, batch_size
    
    image_gen = ImageDataGenerator(
        rescale=1./255, 
        rotation_range=45, 
        zoom_range=0.5, 
        width_shift_range=0.15,
        height_shift_range=0.15, 
        horizontal_flip=True)

    train_data_gen = image_gen.flow_from_directory(
        batch_size=batch_size, 
        directory=train_dir, 
        shuffle=True, 
        target_size=(img_shape,img_shape), 
        class_mode='categorical')

    image_gen_val = ImageDataGenerator(
        rescale=1./255)
    
    val_data_gen = image_gen_val.flow_from_directory(
        batch_size=batch_size, 
        directory=val_dir, 
        shuffle=False, 
        target_size=(img_shape,img_shape), 
        class_mode='categorical')
    
    return train_data_gen, val_data_gen

train_data_gen, val_data_gen = define_image_generators(train_dir, val_dir)

## Deep Learning Model

In [ ]:
learning_rate = 1e-4

def create_model():
    model = Sequential()

    model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D(2, 2))

    model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))

    model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    global learning_rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model

### Training the model

In [ ]:
epochs = # no. of epochs to train the model for
batch_size = 32

def train_model(train_data_gen, val_data_gen):
    
    global epochs, batch_size
    
    model = create_model()
    history = model.fit(train_data_gen, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data=val_data_gen)
    
    return history

Achieved Validation Accuracy: 79.32%, (low but hard to improve upon using simple architectures)

Try using more sophisticated architectures if more compute resources are available

In [ ]:
history = train_model(train_data_gen, val_data_gen)

### Plotting Accuracy and Loss Curves

In [ ]:
def create_plots(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    global epochs
    epochs_range = range(epochs)

    plt.figure(figsize=(6, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
create_plots(history)